In [44]:
pip install openai==0.28

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.1/236.1 MB 1.8 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytesseract

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import xml.etree.ElementTree as ET
from transformers import pipeline
import xml.etree.ElementTree as ET
import re
import cv2
import openai
import numpy as np
import pytesseract

In [4]:
api_key ='sk-T_1Cf61TrcnZiCJH7EGgd70fJHhYxVXNDb2KoVCfaNT3BlbkFJnoKGrG_rAQlf8ImGxyVCp-AYeEmxq7mT01PHK1DGEA'

TASK WAS PERFORMED TO CONVERT NORMAL TEXT TO CORROSPONDING ADB COMMANDS, COORDINATES WERE EXTRACTED FROM XML FILE

In [46]:
# Function to generate an ADB command using a Language Learning Model (LLM)
def generate_adb_command(xml_content, user_prompt, api_key):
    # Set the API key for OpenAI to authenticate the request
    openai.api_key = api_key

    # Create a prompt for the LLM to convert user instructions into corresponding ADB commands
    # The prompt requests the LLM to extract coordinates from the XML data and generate the ADB command based on the user-provided prompt.
    prompt = f"Convert the text '{user_prompt}' to a corresponding ADB command with coordinates. Extract coordinates from the following XML data.\n\nHere is the XML data:\n{xml_content}"

    try:
        # Call the OpenAI API to interact with the LLM
        rehsponse = openai.CatCompletion.create(
            model="gpt-4",  # Specify the LLM model to be used (GPT-4 in this case)
            messages=[
                {"role": "user", "content": prompt}  # Pass the generated prompt to the model
            ],
            max_tokens=200  # Limit the output tokens to prevent overly long responses (adjustable)
        )

        # Extract the text of the ADB command from the API response
        generated_text = response['choices'][0]['message']['content']
        
        # Return the generated command after stripping extra spaces or line breaks
        return generated_text.strip()
    
    # Catch and handle exceptions if the API call fails
    except Exception as e:
        print(f"Error during API call: {e}")  # Print the error message for debugging
        return ""

# Main function to load XML, interact with LLM, and generate the ADB command
def main(xml_file_path, user_prompt, api_key):
    # Open and read the XML file that contains UI layout and element information
    with open(xml_file_path, 'r') as file:
        xml_content = file.read()

    # Generate the ADB command by passing the XML content and user instruction (prompt) to the LLM
    adb_command = generate_adb_command(xml_content, user_prompt, api_key)

    # Output the generated ADB command to the console
    print("Generated ADB Command:")
    print(adb_command)

# Example usage: when the script is executed, the main function is triggered
if __name__ == "__main__":
    xml_file_path = 'cart.xml'  # Path to the XML file containing the UI layout information
    user_prompt = 'open accounts page'  # Instruction provided by the user to be converted into an ADB command
    main(xml_file_path, user_prompt, api_key)  # Call the main function with file path, prompt, and API key


Generated ADB Command:
The command for the operation 'open accounts page' using the given XML data would be:

```
adb shell input tap 756 2328
```

Please note `756` and `2328` are approximate coordinates computed from the bounds of 'Account' button ('android.view.ViewGroup' with content-desc='Account') as per the following:

Bounds for the 'Account' button are `[648,2256][864,2400]`.
X-coordinate would be `(648 + 864)/2 = 756`
Y-coordinate would be `(2256 + 2400)/2 = 2328`


CODE TO VALIDATE WEATHER THE CURRENT PAGE IS SAME OR NOT WHILE COMAPRING THE OLDER SS OF PAGE WITH NEW XML FILE GENERATED AFTER THE CHANGES MADE OVER THE SAME PAGE (BOUNDING BOXES WERE COMPARED AND 70% CONFIDENCE LEVEL WAS TAKEN AS REFERENCE)

In [45]:
import cv2
import re
import xml.etree.ElementTree as ET

# Function to parse the XML and extract UI elements' bounding boxes and text attributes
def parse_xml_for_ui_elements(xml_content):
    ui_elements = []
    
    # Parse the XML content into an element tree structure
    tree = ET.ElementTree(ET.fromstring(xml_content))
    root = tree.getroot()

    # Iterate through each element in the XML structure
    for element in root.iter():
        bounds = element.attrib.get('bounds')  # Get the 'bounds' attribute of the element (representing UI element's position)
        text = element.attrib.get('text', '')  # Get the 'text' attribute of the element (defaults to empty if not found)

        if bounds:
            # Extract the coordinate values from the 'bounds' string (e.g., "[10,20][30,40]") using regular expression
            coords = list(map(int, re.findall(r'\d+', bounds)))
            # Append the UI element's text and bounding box coordinates as a tuple to the list
            ui_elements.append((text, coords))

    # Return the list of UI elements (text and their corresponding bounding boxes)
    return ui_elements

# Function to validate if the screenshot matches the layout described by the XML elements
# This compares UI elements' bounding boxes from the XML and the screenshot to check their alignment
def validate_image_against_xml(screenshot_path, xml_elements):
    # Load the screenshot image from the specified file path
    img = cv2.imread(screenshot_path)
    
    # Convert the screenshot to grayscale for processing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect edges in the image using OpenCV's Canny edge detection (for placeholder detection)
    edges = cv2.Canny(gray, 50, 150)

    # Loop through each UI element extracted from the XML
    for text, coords in xml_elements:
        x1, y1, x2, y2 = coords  # Extract the bounding box coordinates (top-left and bottom-right corners)
        
        # Draw a green rectangle around the expected position of each UI element on the screenshot
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # (x1, y1) is top-left, (x2, y2) is bottom-right

    # Use template matching to validate that the detected UI elements in the image match the XML layout more precisely
    for text, coords in xml_elements:
        x1, y1, x2, y2 = coords  # Extract coordinates again
        
        # Define the region of interest (ROI) in the screenshot, corresponding to the UI element's position
        roi = gray[y1:y2, x1:x2]

        # Use template matching to compare the region in the image to its expected location
        match = cv2.matchTemplate(gray, roi, cv2.TM_CCOEFF_NORMED)

        # Extract the matching confidence values (min/max) from the template matching result
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(match)

        # If the maximum confidence value (max_val) is below the threshold (e.g., 0.7), consider it a mismatch
        if max_val < 0.7:
            print(f"Element '{text}' does not match the screenshot layout.")  # Log which element failed
            return False  # Return False if any element does not match the expected position

    return True  # Return True if all UI elements match their expected layout

# Main function to execute the script
def main(xml_file_path, screenshot_path):
    # Read the XML file content
    with open(xml_file_path, 'r') as file:
        xml_content = file.read()

    # Parse the XML to extract UI elements and their bounding boxes
    xml_elements = parse_xml_for_ui_elements(xml_content)

    # Validate the screenshot layout against the parsed XML elements
    is_valid = validate_image_against_xml(screenshot_path, xml_elements)

    # Output the result of the validation
    print("Image Validation Result:")
    print("Valid Image!" if is_valid else "Image does not match the expected layout.")

# Example usage
if __name__ == "__main__":
    xml_file_path = 'cart_updated.xml'  # Path to the XML file containing the UI layout information
    screenshot_path = 'cart.png'  # Path to the screenshot to validate
    main(xml_file_path, screenshot_path)


Element '' does not match the screenshot layout.
Image Validation Result:
Image does not match the expected layout.
